In [1]:
import pandas as pd
import numpy as np
import datetime
import math as m
from sklearn.impute import KNNImputer,SimpleImputer
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import re
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',200)

#### ya hice un pkl para no correr todo

In [2]:
df=pd.read_csv("US_Accidents_June20.csv")#.sample(n=100000, random_state=1)

In [ ]:
print('Rows     :',df.shape[0])
print('Columns  :',df.shape[1])
print('\nFeatures :\n     :',df.columns.tolist())
print('\nMissing values    :',df.isnull().values.sum())
print('\nUnique values :  \n',df.nunique())

In [6]:
df['Start_Time']=pd.to_datetime(df['Start_Time'])
df['End_Time']=pd.to_datetime(df['End_Time'])

In [7]:
df["time"]=df["End_Time"]-df["Start_Time"]

In [9]:
df["TimeMinutes"]= pd.to_timedelta(df["time"]).astype('timedelta64[m]').astype(int)

In [ ]:
df['Hour']=df['Start_Time'].dt.hour
df['Weekday']=df['Start_Time'].dt.strftime('%a')
df['Year']=df['Start_Time'].dt.year
df['Month']=df['Start_Time'].dt.strftime('%b')
df['Day']=df['Start_Time'].dt.day

In [10]:
df1=df[df["TimeMinutes"]>0].copy()

In [11]:
df1=df1.reset_index(drop=True)

In [ ]:
County=df[["County","ID"]].groupby("County").count()
County.rename(columns={"ID":"County_Freq"},inplace=True)
Zipcode=df[["Zipcode","ID"]].groupby("Zipcode").count()
Zipcode.rename(columns={"ID":"Zipcode_Freq"},inplace=True)
State=df[["State","ID"]].groupby("State").count()
State.rename(columns={"ID":"State_Freq"},inplace=True)
City=df[["City","ID"]].groupby("City").count()
City.rename(columns={"ID":"City_Freq"},inplace=True)
Airport=df[["Airport_Code","ID"]].groupby("Airport_Code").count()
Airport.rename(columns={"ID":"Airport_Freq"},inplace=True)

In [ ]:
df1=pd.merge(df1,County,how="left", left_on="County", right_on="County")
df1=pd.merge(df1,State,how="left", left_on="State", right_on="State")
df1=pd.merge(df1,City,how="left", left_on="City", right_on="City")
df1=pd.merge(df1,Airport,how="left", left_on="Airport_Code", right_on="Airport_Code")

# seccionamos el df
##### no fue necesario de momento

In [ ]:
df1["Source"].value_counts()

In [ ]:
df_B=df1[df1["Source"]=="Bing"].copy()
df_MQ=df1[df1["Source"]=="MapQuest"].copy()
df_BMQ=df1[df1["Source"]=="MapQuest-Bing"].copy()

In [ ]:
df_B=df_B.reset_index(drop=True)
df_MQ=df_MQ.reset_index(drop=True)
df_BMQ=df_BMQ.reset_index(drop=True)

## revisamos outliers

In [ ]:
df_B.describe(list(np.arange(.1,1,.1))+[.05,.95])## windspeed

In [ ]:
df_MQ.describe(list(np.arange(.1,1,.1))+[.05,.95])## windspeed

In [ ]:
df_BMQ.describe(list(np.arange(.1,1,.1))+[.05,.95])## windspeed

##### Como las 3 tienen el outlier en windspeed y notamos que lo podemos quitar de la tabla original procedemos con eso

In [12]:
q1=df1["Wind_Speed(mph)"].quantile(.25)
q3=df1["Wind_Speed(mph)"].quantile(.75)

In [13]:
IQR=1.5*(q3-q1)

In [ ]:
q3+IQR

In [ ]:
df1["Wind_Speed(mph)"].min()

In [14]:
df1=df1[(df1["Wind_Speed(mph)"]>=q1-IQR)&(df1["Wind_Speed(mph)"]<=q3+IQR)].copy()

In [15]:
df1=df1.reset_index(drop=True)

# Reducción de dimensiones parte 1

In [ ]:
df1.shape

###### Eliminamos columnas con demasiados nulos, porque los métodos de imputacion no serviran

In [16]:
df1.isnull().sum()

ID                             0
Source                         0
TMC                       907122
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  2103589
End_Lng                  2103589
Distance(mi)                   0
Description                    1
Number                   1936107
Street                         0
Side                           0
City                          97
County                         0
State                          0
Zipcode                        0
Country                        0
Timezone                       0
Airport_Code                   0
Weather_Timestamp              0
Temperature(F)              9233
Wind_Chill(F)            1395044
Humidity(%)                12497
Pressure(in)                3338
Visibility(mi)              9924
Wind_Direction                43
Wind_Speed(mph)                0
Precipitat

In [17]:
dropear=["End_Lat","End_Lng","Number","Wind_Chill(F)","Precipitation(in)",
         "Weather_Timestamp","time","Sunrise_Sunset","Civil_Twilight"
         ,"Nautical_Twilight","Astronomical_Twilight","ID"]

In [18]:
df1[dropear].isnull().sum()/df1.shape[0]

End_Lat                  0.698702
End_Lng                  0.698702
Number                   0.643073
Wind_Chill(F)            0.463360
Precipitation(in)        0.528233
Weather_Timestamp        0.000000
time                     0.000000
Sunrise_Sunset           0.000033
Civil_Twilight           0.000033
Nautical_Twilight        0.000033
Astronomical_Twilight    0.000033
ID                       0.000000
dtype: float64

In [ ]:
df1=df1.drop(dropear,axis=1)

In [ ]:
df1.columns

#### Revisamos variables Unarias

In [ ]:
print('\nUnique values :  \n',df1.nunique())

In [19]:
unarias=["Turning_Loop","Bump","Give_Way","No_Exit","Railway","Roundabout",
         "Traffic_Calming","Country"]

In [32]:
df1["Turning_Loop"].value_counts(1)

False    1.0
Name: Turning_Loop, dtype: float64

In [20]:
df1["Amenity"].value_counts(1)

False    0.988008
True     0.011992
Name: Amenity, dtype: float64

In [21]:
df1["Bump"].value_counts(1)

False    0.999828
True     0.000172
Name: Bump, dtype: float64

In [22]:
df1["Crossing"].value_counts(1)

False    0.920271
True     0.079729
Name: Crossing, dtype: float64

In [23]:
df1["Give_Way"].value_counts(1)

False    0.997344
True     0.002656
Name: Give_Way, dtype: float64

In [24]:
df1["Junction"].value_counts(1)

False    0.9195
True     0.0805
Name: Junction, dtype: float64

In [25]:
df1["No_Exit"].value_counts(1)

False    0.998738
True     0.001262
Name: No_Exit, dtype: float64

In [26]:
df1["Railway"].value_counts(1)

False    0.991251
True     0.008749
Name: Railway, dtype: float64

In [27]:
df1["Roundabout"].value_counts(1)

False    0.999947
True     0.000053
Name: Roundabout, dtype: float64

In [28]:
df1["Station"].value_counts(1)

False    0.98004
True     0.01996
Name: Station, dtype: float64

In [29]:
df1["Stop"].value_counts(1)

False    0.98529
True     0.01471
Name: Stop, dtype: float64

In [30]:
df1["Traffic_Calming"].value_counts(1)

False    0.999608
True     0.000392
Name: Traffic_Calming, dtype: float64

In [31]:
df1["Traffic_Signal"].value_counts(1)

False    0.819378
True     0.180622
Name: Traffic_Signal, dtype: float64

In [ ]:
df1=df1.drop(unarias,axis=1)

In [ ]:
df1.columns

## Limpieza de variables, con un diccionario

In [ ]:
df1["Wind_Direction"].unique()

In [ ]:
## WSW WEST SOUTHWEST,

In [ ]:
dic_viento={"CALM":"calm","South":"S","West":"W","North":"N","VAR":"Variable","East":"E",
           "SSW":"SW","WSW":"SW","WNW":"NW","NNW":"NW","NNE":"NE","SSE":"SE","ESE":"SE",
           "ENE":"NE"}

In [ ]:
df1["Wind_Direction"]=df1["Wind_Direction"].replace(dic_viento)

In [ ]:
df1["Wind_Direction"].value_counts(1,dropna=False)

In [ ]:
df1['Weather_Condition'].value_counts()

In [ ]:
# show distinctive weather conditions 
weather =' '.join(df1['Weather_Condition'].dropna())
#weather = np.unique(np.array(re.split("!|\s/\s|\sand\s|\swith\s|Partly\s|Mostly\s|Blowing\s|Freezing\s", weather))).tolist()
#print("Weather Conditions: ", weather)


In [ ]:
type(weather)

In [ ]:
bad_chars = [';', ':', '!', "*","-","/"]
for i in bad_chars :
    test_string = weather.replace(i, '')

In [ ]:
wordlist = test_string.split()

In [ ]:
from collections import Counter
Counter(wordlist)

In [ ]:
df1['Test'] = np.where(df1['Weather_Condition'].str.contains('Cloud', case=False, na = False),'Cloudy', df1['Weather_Condition'])
df1['Test'] = np.where(df1['Test'].str.contains('thunder', case=False, na = False),'ThunderStorm', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('T-Storm', case=False, na = False),'ThunderStorm', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Sand', case=False, na = False),'Dust Whirls', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Whirls', case=False, na = False),'Dust Whirls', df1['Test'])

In [ ]:
df1['Test'] = np.where(df1['Test'].str.contains('Fog', case=False, na = False),'Fog', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('snow', case=False, na = False),'snow', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Rain', case=False, na = False),'rainy', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Drizzle', case=False, na = False),'rainy', df1['Test'])

In [ ]:
df1['Test'] = np.where(df1['Test'].str.contains('Hail', case=False, na = False),'Hail', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Haze', case=False, na = False),'Haze', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Windy', case=False, na = False),'Windy', df1['Test'])

In [ ]:
df1['Test'] = np.where(df1['Test'].str.contains('Smoke', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Windy', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Wintry', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Mist', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Showers', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Precipitation', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Hail', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Squalls', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Tornado', case=False, na = False),'Other', df1['Test'])

In [ ]:
df1['Test'] = np.where(df1['Test'].str.contains('Pellets', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Sleet', case=False, na = False),'Other', df1['Test'])
df1['Test'] = np.where(df1['Test'].str.contains('Dust', case=False, na = False),'Other', df1['Test'])

In [ ]:
df1=df1.rename(columns={"Test": "Weather"})

In [ ]:
weather =' '.join(df1['Test'].dropna())

In [ ]:
bad_chars = [';', ':', '!', "*","-","/",'of','in','the','and','Low','Nearby']
for i in bad_chars :
    test_string = weather.replace(i, '')

In [ ]:
wordlist = test_string.split()
Counter(wordlist)

In [ ]:
df1["Weather"].value_counts(1)

### last check

In [ ]:
#df1.to_pickle("dummy.pkl")

In [ ]:
df1=pd.read_pickle("dummy.pkl")### de las discretas solo falta validar que quedó bien weather

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:
df1["Day"].value_counts(1,dropna=False)

In [ ]:
df1["Zipcode"].nunique()

In [ ]:
revisar=["Temperature(F)","Humidity(%)","Pressure(in)","Visibility(mi)","Wind_Speed(mph)"]
revisar2=["Distance(mi)","TimeMinutes"]

In [ ]:
df1[revisar].describe(list(np.arange(.1,1,.1))+[.05,.95]+[.01,.99])

In [ ]:
q3=df1["Visibility(mi)"].quantile(.75)
q1=df1["Visibility(mi)"].quantile(.25)

In [ ]:
IQR=1.5*(q3-q1)

In [ ]:
q3+IQR

In [ ]:
df2=df1[df1["Visibility(mi)"]<=10].copy()

In [ ]:
df2=df2.reset_index(drop=True)

In [ ]:
df2[revisar].describe(list(np.arange(.1,1,.1))+[.05,.95]+[.01,.99])

In [ ]:
df2.shape

In [ ]:
df1.shape

In [ ]:
2984488/3513617

In [ ]:
revision_clima=df2[['Weather_Condition','Weather']]

In [ ]:
revision_clima['Weather'].unique()

In [ ]:
revision_clima.loc[revision_clima['Weather']=='Other', ['Weather_Condition']].value_counts()

In [ ]:
#df2.to_pickle("Tabla_Proyecto_Limpia.pkl")

###### HASTA AQUÍ

### separar mapquest y bing https://www.kaggle.com/jingzongwang/usa-car-accidents-severity-prediction

In [ ]:
df_B=df1[df1["Source"]=="Bing"].copy()
df_MQ=df1[df1["Source"]=="MapQuest"].copy()
df_BMQ=df1[df1["Source"]=="MapQuest-Bing"].copy()

In [ ]:
df_B=df_B.reset_index(drop=True)
df_MQ=df_MQ.reset_index(drop=True)
df_BMQ=df_BMQ.reset_index(drop=True)

In [ ]:
df1.columns

### Escala de variables

In [ ]:
sc_B=StandardScaler()
sc_MQ=StandardScaler()
sc_BMQ=StandardScaler()

In [ ]:
continuas=['Distance(mi)','Temperature(F)', 'Humidity(%)','Pressure(in)', 'Visibility(mi)'
           ,'Wind_Speed(mph)','TimeMinutes',]

In [ ]:
sc_B.fit(df_B[continuas])
sc_MQ.fit(df_MQ[continuas])
sc_BMQ.fit(df_BMQ[continuas])

In [ ]:
X1=sc_B.transform(df_B[continuas])
X2=sc_MQ.transform(df_MQ[continuas])
X3=sc_BMQ.transform(df_BMQ[continuas])

In [ ]:
X1=pd.DataFrame(X1,columns=df_B[continuas].columns)
X2=pd.DataFrame(X2,columns=df_MQ[continuas].columns)
X3=pd.DataFrame(X3,columns=df_BMQ[continuas].columns)

In [ ]:
X1.columns = [str(col) + '_Escalado' for col in X1.columns]
X2.columns = [str(col) + '_Escalado' for col in X2.columns]
X3.columns = [str(col) + '_Escalado' for col in X3.columns]

In [ ]:
df_B=pd.concat([df_B,X1],axis=1)
df_MQ=pd.concat([df_MQ,X2],axis=1)
df_BMQ=pd.concat([df_BMQ,X3],axis=1)

In [ ]:
df_MQ.isnull().sum()

## Imputación de variables

In [ ]:
def get_value_off_missing_data(data_original,data_impute):
#     f = plt.subplots()    
    sns.distplot(data_original.dropna(),label="datos originales",hist=False,kde_kws={"fill":True},)
    sns.distplot(data_impute,color="red",label="datos imputados",hist=False,kde_kws={"fill":True},)
    plt.legend()

### Multivariante

In [ ]:
imputar_mult=['Distance(mi)_Escalado', 'Temperature(F)_Escalado',
       'Humidity(%)_Escalado', 'Pressure(in)_Escalado',
       'Visibility(mi)_Escalado', 'Wind_Speed(mph)_Escalado',
       'TimeMinutes_Escalado']

In [ ]:
imputador = KNNImputer(n_neighbors=7)

In [ ]:
imputador.fit(df2[imputar_mult])

In [ ]:
data_knn_imputer=pd.DataFrame(imputador.transform(df2[imputar_mult]),columns=imputar_mult)

In [ ]:
get_value_off_missing_data(df2[imputar_mult],data_knn_imputer[imputar_mult])

### Univariante

In [ ]:
## imputar side por moda y wind direction y weather, quitar zip code

# PCA

In [ ]:
df1.head()